En kaggle

In [ ]:
!pip install ultralytics opencv-python --quiet

In [ ]:
!pip install open_clip_torch

In [ ]:
#lap for video

import math
import yaml
import csv

Carga modelo

In [ ]:
# Ruta a tu modelo entrenado
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np

MODEL_PATH = "/kaggle/input/yolov8-sku110k/pytorch/default/1/best.pt"

model = YOLO(MODEL_PATH)

Inicializa dataframe para guardar resultados

In [ ]:
import pandas as pd

# -------------------------------
# Inicializar DataFrame global vacío
# -------------------------------
df_inventory_global = pd.DataFrame()

print("✅ DataFrame global inicializado:")
df_inventory_global


Inferencia del modelo

In [ ]:
# Imagen de prueba
IMAGE_PATH = "/kaggle/input/sku110k-annotations/SKU110K_fixed/images/test/test_0.jpg"

# Cargar imagen
image = cv2.imread(IMAGE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Inferencia
results = model(image)
boxes = results[0].boxes

print("📦 Total de objetos detectados:", len(boxes))

Simula LVM con productos aleatorios para para probar pipeline: imagen -> deteccion -> lvm -> dataframe

TODO: Sustituir por modelo ViT-B/32

In [ ]:
import random

# -------------------------------
# LVM simulado: asigna producto aleatorio
# -------------------------------
def classify_product(roi):
    # Retorna producto aleatorio de la lista
    products = ["coke_can", "pringles", "evian_water", "snickers", "lays"]
    return random.choice(products)

Procesa deteccion y almacena usando etiqueta de lvm simulado

In [ ]:
# -------------------------------
# Procesar detecciones y almacenar productos
# -------------------------------
detected_products = []
img_draw = image.copy()

for i, box in enumerate(boxes):
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
    roi = image[y1:y2, x1:x2]

    product_name = classify_product(roi)
    detected_products.append(product_name)

    # Dibujar bounding box con etiqueta
    cv2.rectangle(img_draw, (x1, y1), (x2, y2), (0,255,0), 2)
    cv2.putText(img_draw, product_name, (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

# -------------------------------
# Mostrar imagen con detecciones
# -------------------------------
plt.figure(figsize=(12,12))
plt.imshow(img_draw)
plt.axis("off")
plt.show()

# -------------------------------
# Crear fila de inventario con conteos por producto + timestamp
# -------------------------------
from collections import Counter
from datetime import datetime

counts = Counter(detected_products)
counts["timestamp"] = datetime.now()

# Añadir columnas nuevas al DataFrame global dinámicamente
for col in counts:
    if col not in df_inventory_global.columns:
        # Insertar la nueva columna justo antes de la última columna
        df_inventory_global.insert(len(df_inventory_global.columns)-1, col, 0)

# Añadir la fila
df_inventory_global = pd.concat([df_inventory_global, pd.DataFrame([counts])], ignore_index=True)


# -------------------------------
# Limpiar tipos y NaN
# -------------------------------
# Reemplazar NaN por 0 y asegurar que las columnas de productos sean enteros
for col in df_inventory_global.columns:
    if col != "timestamp":
        # Convertir a numérico, NaN -> 0, luego a int
        df_inventory_global[col] = pd.to_numeric(df_inventory_global[col], errors='coerce').fillna(0).astype(int)

df_inventory_global


Simula DB con productos decrecientes para probar graficas

In [ ]:
# -------------------------------
# Simulate a "realistic" decreasing inventory
# -------------------------------

import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

products = ["coke_can", "pringles", "evian_water", "snickers", "lays"]

# Number of snapshots
num_snapshots = 5
base_time = datetime.now() - timedelta(days=1)

snapshots = []

# Step 1: Initialize starting counts
start_counts = {p: random.randint(15, 25) for p in products}

# Step 2: Generate decreasing counts over time
for i in range(num_snapshots):
    timestamp = base_time + timedelta(hours=i*6)
    snapshot = {}
    for p in products:
        # Decrease by random amount between 0 and 5, but never below 0
        if i == 0:
            snapshot[p] = start_counts[p]
        else:
            snapshot[p] = max(snapshots[-1][p] - random.randint(0, 5), 0)
    snapshot["timestamp"] = timestamp
    snapshots.append(snapshot)

# Convert to DataFrame
db_mock = pd.DataFrame(snapshots)
print("Simulated 'decreasing' DB:")
db_mock

In [ ]:
# -------------------------------
# Cell 2: Load inventory variables
# -------------------------------

# Choose two snapshots to compare
snapshot1 = db_mock.iloc[0]  # earliest snapshot
snapshot2 = db_mock.iloc[-1] # latest snapshot

# Convert to dictionaries for plotting
inventory1 = {p: snapshot1[p] for p in products}
inventory2 = {p: snapshot2[p] for p in products}

# Save timestamps as metadata
timestamp1 = snapshot1["timestamp"]
timestamp2 = snapshot2["timestamp"]

print("Inventory 1:", inventory1)
print("Inventory 2:", inventory2)
print("Timestamps:", timestamp1, "->", timestamp2)


In [ ]:
# -------------------------------
# Cell 3: Print graphs
# -------------------------------

import matplotlib.pyplot as plt

# --- Graph 1: Counts per class ---
x = range(len(products))

plt.figure(figsize=(10, 5))
plt.bar([i-0.15 for i in x], [inventory1[p] for p in products], width=0.3, label=f"T1: {timestamp1}")
plt.bar([i+0.15 for i in x], [inventory2[p] for p in products], width=0.3, label=f"T2: {timestamp2}")
plt.xticks(x, products)
plt.ylabel("Item Count")
plt.title("Product Counts per Class")
plt.legend()
plt.show()

# --- Graph 2: Items added / removed ---
diff = {p: inventory2[p] - inventory1[p] for p in products}
colors = ['green' if v >= 0 else 'red' for v in diff.values()]

plt.figure(figsize=(10, 5))
plt.bar(products, diff.values(), color=colors)
plt.axhline(0, color='black', linewidth=0.8)
plt.ylabel("Items Added/Removed")
plt.title(f"Inventory Changes from {timestamp1} to {timestamp2}")
plt.show()

# -------------------------------
# Line plot: Evolution of products over time
# -------------------------------

# Extract timestamps
timestamps = db_mock["timestamp"]

plt.figure(figsize=(12, 6))

# Plot each product as a line
for product in products:
    plt.plot(timestamps, db_mock[product], marker='o', label=product)

plt.xlabel("Timestamp")
plt.ylabel("Item Count")
plt.title("Inventory Evolution Over Time")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd

threshold = 3  # stock shortage threshold

# Convert timestamps to numeric values (hours since first snapshot)
X = np.array([(t - db_mock["timestamp"].iloc[0]).total_seconds()/3600 for t in db_mock["timestamp"]]).reshape(-1, 1)

for product in products:
    y = db_mock[product].values

    plt.figure(figsize=(8, 5))

    # Plot actual inventory points
    plt.plot(db_mock["timestamp"], y, 'o-', label=f"{product} (actual)")

    # Fit linear regression
    model = LinearRegression()
    model.fit(X, y)

    # Predict future trend (extend 24 hours beyond last snapshot)
    future_hours = np.linspace(0, X[-1][0]+24, 100).reshape(-1, 1)
    y_pred = model.predict(future_hours)
    future_times = [db_mock["timestamp"].iloc[0] + pd.Timedelta(hours=h[0]) for h in future_hours]

    # Plot predicted line
    plt.plot(future_times, y_pred, '--', label=f"{product} (predicted)")

    # Find crossing point with threshold
    if model.coef_[0] != 0:
        hours_to_threshold = (threshold - model.intercept_) / model.coef_[0]
        if hours_to_threshold > 0:
            t_cross = db_mock["timestamp"].iloc[0] + pd.Timedelta(hours=hours_to_threshold)
            plt.plot(t_cross, threshold, 'rx', markersize=10)
            plt.text(t_cross, threshold+0.5, f"Predicted shortage", color='red')

    # Draw threshold line
    plt.axhline(y=threshold, color='gray', linestyle='--', label=f"Threshold = {threshold}")

    plt.xlabel("Timestamp")
    plt.ylabel("Item Count")
    plt.title(f"Inventory Evolution and Prediction: {product}")
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Modelo preentrenado
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Imagen de ejemplo (máscara segmentada por SAM)
image = Image.open("mask_coca_can.jpg")

# Categorías posibles
categories = ["coca_can", "pringles", "evian_water"]

# Procesamiento
inputs = processor(text=categories, images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)

# logits_per_image → predicción de la categoría
logits = outputs.logits_per_image
predicted_idx = logits.argmax().item()
print("Categoría predicha:", categories[predicted_idx])
